# Import libraries

In [1]:
import os
import math
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from copy import deepcopy
from keras.layers import *
from keras.layers import LSTM
from keras.layers import Dense
from datetime import datetime
from keras.layers import Dropout
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error,r2_score

# Loading Data

In [4]:
#df=pd.read_csv('../MPT Data/Combined.csv')
df=pd.read_csv('../MPT Data/Combined.csv')
df.drop(['TRADE_ENTRY_DATE','Unnamed: 0'],axis=1,inplace=True)
for _,columns in enumerate(df):
    df[[columns]]=df[[columns]].astype('float')
df.columns=['ATRL','bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML',
      'OGDC','PPL','PSO','SEARL','TRG']
df.head()

,ATRL,bahl,EPCL,FCCL,LUCK,MCB,MLCF,NETSOL,NML,OGDC,PPL,PSO,SEARL,TRG
0,82.90,26.50,22.75,6.70,51.35,144.00,5.00,17.40,31.01,73.85,176.75,200.80,64.05,1.37
1,83.30,26.58,23.00,6.70,51.25,144.17,4.98,17.45,31.30,74.00,177.00,200.99,64.00,1.40
2,85.00,26.90,22.90,6.75,51.50,144.49,5.05,17.40,31.20,73.70,177.30,201.03,63.10,1.38
3,85.40,27.02,23.19,6.70,51.85,145.00,5.08,17.50,31.05,73.80,177.33,201.12,60.80,1.31
4,86.85,28.10,23.11,6.74,51.85,145.00,5.17,17.26,30.80,73.70,176.80,201.45,62.00,1.40


# Trading Framework

## Trader function

In [5]:
def trader(X,nS3,share_price1,share_price2):
    '''
    This function takes cash,Number of shares
    Share price at minimum and share price at maximum
    and returns the remaining amount
    
    We are selling 20pc of stock at maximum price and buy
    it at minimum point
    '''
    #Selling shares at maximum point in 100 entries
    Amount_Gained=((nS3*share_price2)*.2)
    Brocker_fee=((nS3*share_price2)*.2)*.3*.01
    nS3=nS3-(nS3*.2)
    X=X+Amount_Gained-Brocker_fee
    #Buying shares at minimum point in 100 entries
    Shares_bought=(.997*Amount_Gained)/share_price1
    nS3=nS3+Shares_bought
    X=X-(.997*Amount_Gained)
    return X,math.floor(nS3)

## Portfolio declaration

In [6]:
lm=250
X=2000
tpf=98000
TN=X+tpf
df1=df
shares1={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
Min={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
Max={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
for share in shares1:
    shares1[share]=math.floor(7000/df1.loc[0,share])

## Testing Framework

In [ ]:
TN=[100000]
df2=df
for i in range(df2.shape[0]//lm-1):
    df1=df2[i*lm:(i*lm)+lm]
    for share in shares1:
        Min[share]=df1[share].idxmin()
        Max[share]=df1[share].idxmax()
    for share in shares1:
        if Min[share]!=Max[share]:
           X,shares1[share]=trader(X,shares1[share],df2.loc[Min[share],share],df2.loc[Max[share],share])
           tpf+=(df2.loc[(i*lm)+lm,share]*shares1[share])
           TN.append(X+tpf)
           print(X,tpf)
    tpf=0
            
    

# Preprocessing

## Train Test split

In [7]:
train_data=[]
test_data=[]
for share in shares1:
    train_data.append(df.loc[:299999,share])
    test_data.append(df.loc[300000:400000,share])


### Normalizing Data
Now you need to define a scaler to normalize the data. MinMaxScalar scales all the data to be in the region of 0 and 1. You can also reshape the training and test data to be in the shape [data_size, num_features]

In [8]:
scaler = MinMaxScaler()
for i in range(len(shares1)):
    train_data[i] = train_data[i].values.reshape(-1,1)
    test_data[i] = test_data[i].values.reshape(-1,1)
td=train_data.copy()
tr=test_data.copy()

Due to the observation we made earlier, that is, different time periods of data have different value ranges, you normalize the data by splitting the full series into windows. If you don't do this, the earlier data will be close to 0 and will not add much value to the learning process. Here you choose a window size of 25000.


In [103]:
# Train the Scaler with training data and smooth data
t_norm=[]
train_data_norm=[]
test_data_norm=[]
smoothing_window_size = 200
test_scalers,train_scalers=[],[]
for i in range(len(shares1)):
    t_norm=[]
    train_data_acq=td[i].copy() 
    test_data_acq=tr[i].copy()
    for di in range(0,300000,smoothing_window_size):
        scaler=MinMaxScaler()
        scaler.fit(train_data_acq[di:di+smoothing_window_size,:])
        train_data_acq[di:di+smoothing_window_size,:] = scaler.transform(train_data_acq[di:di+smoothing_window_size,:])
        t_norm.append(deepcopy(scaler))
    train_data_norm.append(train_data_acq)
    train_scalers.append(t_norm.copy())
#Normalize the test data too
    sc=MinMaxScaler()
    test_data_norm.append(sc.fit(test_data_acq).transform(test_data_acq).reshape(-1))
    test_scalers.append(deepcopy(sc))
# Reshape both train and test data
for i in range(len(shares1)):
    train_data_norm[i] = train_data_norm[i].reshape(-1)

## Preparing Train and test data for training

### Train data

#### Normalized Train data

In [10]:
Xt=[]
yt=[]
X_train_norm = []
y_train_norm = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(train_data_norm[j])):
        Xt.append(train_data_norm[j][i-60:i])
        yt.append(train_data_norm[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_train_norm.append(Xt)
    y_train_norm.append(yt)

###  Non Normalized Test Data

In [11]:
Xt=[]
yt=[]
X_train_comb = []
y_train_comb = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(train_data_norm[j])):
        Xt.append(train_data[j][i-60:i])
        yt.append(train_data[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_train_comb.append(Xt)
    y_train_comb.append(yt)

### Test data

#### Non Normalized Test Data

In [12]:
Xt=[]
yt=[]
X_test_comb = []
y_test_comb = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(test_data[j])):
        Xt.append(test_data[j][i-60:i])
        yt.append(test_data[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_test_comb.append(Xt)
    y_test_comb.append(yt)

### Normalized Test Data

In [13]:
Xt=[]
yt=[]
X_test_norm = []
y_test_norm = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(test_data[j])):
        Xt.append(test_data_norm[j][i-60:i])
        yt.append(test_data_norm[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_test_norm.append(Xt)
    y_test_norm.append(yt)

# Machine Learning Model 

## Model Declaration

In [14]:
def model():
    regressor = Sequential()

    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (60, 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    
    return regressor

## Storing initialization of 14 models for 14 stocks and MAPE

In [17]:
models=[]
MAPE={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDX': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
MAP=['ATRL','bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML','OGDX',
       'PPL','PSO','SEARL','TRG']
for share in shares1:
    models.append(model()) 

## Training

In [141]:
for i in range(14):
    models[i].fit(X_train_norm[i],y_train_norm[i],epochs=100,batch_size=10000) 

Epoch 1/100
30/30 [==============================] - 10s 322ms/step - loss: 0.0074
Epoch 2/100
30/30 [==============================] - 10s 326ms/step - loss: 0.0074
Epoch 3/100
30/30 [==============================] - 10s 331ms/step - loss: 0.0074
Epoch 4/100
30/30 [==============================] - 10s 330ms/step - loss: 0.0074
Epoch 5/100
30/30 [==============================] - 10s 327ms/step - loss: 0.0074
Epoch 6/100
30/30 [==============================] - 10s 325ms/step - loss: 0.0073
Epoch 7/100
30/30 [==============================] - 10s 323ms/step - loss: 0.0074
Epoch 8/100
30/30 [==============================] - 10s 323ms/step - loss: 0.0073
Epoch 9/100
30/30 [==============================] - 10s 325ms/step - loss: 0.0073
Epoch 10/100
30/30 [==============================] - 10s 328ms/step - loss: 0.0073
Epoch 11/100
30/30 [==============================] - 10s 325ms/step - loss: 0.0073
Epoch 12/100
30/30 [==============================] - 10s 327ms/step - loss: 0.0073
E

## Saving Model Weights

In [22]:
for i in range(14):
  models[i].save(MAP[i]+'_model')


KeyboardInterrupt: 

## Loading Models

In [23]:
for i in range(14):
  print('loaded model :',MAP[i])
  models[i]=keras.models.load_model('Models/'+MAP[i]+'_model')

loaded model : ATRL
loaded model : bahl
loaded model : EPCL
loaded model : FCCL
loaded model : LUCK
loaded model : MCB
loaded model : MLCF
loaded model : NETSOL
loaded model : NML
loaded model : OGDX
loaded model : PPL
loaded model : PSO
loaded model : SEARL
loaded model : TRG


# MAPE for LSTM models

## MAPE Calculator

In [25]:
def mape(yhat,y_test):  
  arr=np.array([[]])
  for i in range(len(yhat)):
      arr1=np.array([np.abs((y_test_comb[0][i] - yhat[i]) / yhat[i])]) 
      arr=np.concatenate((arr,arr1),axis=1)
  return arr.mean()


## Calculating MAPE

In [28]:
yhat_for_all_models=[]
for i in range(14):
  yhat=models[i].predict(X_test_norm[i],batch_size=5000)
  yhat_for_all_models.append(test_scalers[i].inverse_transform(yhat))
  print('Done :',i)

Done : 0
Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13


## Saving MAPE

In [36]:
for i in range(14):
    List=[]
    List.append(mape(yhat_for_all_models[i],y_test_comb[i]))
    MAPE[MAP[i]]=List
df_MAPE=pd.DataFrame(MAPE)
df_MAPE.to_csv('MAPE.csv')

# Putting Model in Trading Framework

## Getting Prediction for train data

In [107]:
yhat_for_train=[]

for i in range(14):
  yt=[]
  yhat=models[i].predict(X_train_norm[i],batch_size=5000)
  for j in range(len(train_scalers[i])):
      yt.extend(train_scalers[i][j].inverse_transform(yhat[200*j:(200*j)+200]))
  yhat_for_train.append(yt)
  print('Done :',i) 

Done : 0
Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13


In [124]:
df_train=pd.DataFrame(yhat_for_train[0])
for i in range(1,14):
    df_temp=pd.DataFrame(yhat_for_train[i])
    df_train=pd.concat([df_train,df_temp],axis=1)
df_train

,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,85.495445,28.712223,24.044535,7.060798,54.012886,143.909180,5.060462,17.288521,31.123518,73.995895,173.974640,199.638260,49.915535,1.642860
1,85.276649,28.628889,23.997244,7.035091,53.624226,143.681442,5.099741,17.318390,31.065128,74.010178,173.765945,199.617630,50.199841,1.642868
2,85.119873,28.738647,23.855869,7.031153,53.951996,143.649918,5.077675,17.355007,31.153982,73.996681,173.860306,199.661713,50.309097,1.642910
3,85.089920,29.171711,23.800381,7.073793,54.362797,143.720032,5.083728,17.424330,31.173611,74.114754,174.182144,199.311737,51.082546,1.606955
4,85.004517,29.383968,23.788126,7.097402,54.425026,143.923843,5.053038,17.501820,31.224255,74.345009,174.749466,199.111786,50.506191,1.596442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299935,217.625076,84.500397,33.630356,17.507677,433.524689,190.203186,31.509790,76.900787,170.038574,175.923737,110.916931,357.105408,210.007874,42.312984
299936,217.695312,84.500397,33.630356,17.507677,433.485229,190.169022,31.509790,76.900787,169.912079,175.921112,110.839447,357.100494,210.007874,42.312408
299937,217.541809,84.500397,33.630356,17.507677,433.448944,190.166946,31.509790,76.900787,169.866364,175.929886,110.752228,357.098480,210.007874,42.312199
299938,217.508057,84.500397,33.630356,17.507677,433.416718,190.158936,31.509790,76.900787,169.646072,175.923080,110.967163,356.789459,210.007874,42.253231


## Getting Prediction for test data

In [ ]:
yhat_for_all_models=[]
for i in range(14):
  yhat=models[i].predict(X_test_norm[i],batch_size=5000)
  yhat_for_all_models.append(test_scalers[i].inverse_transform(yhat))
  print('Done :',i)

In [128]:
df1=pd.DataFrame(yhat_for_all_models[0])
for i in range(1,14):
    df2=pd.DataFrame(yhat_for_all_models[i])
    df1=pd.concat([df1,df2],axis=1)
df1

,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,214.642029,84.500397,33.630356,17.507677,442.986328,187.670898,31.50979,76.900787,168.380676,178.574905,114.264664,355.441254,210.007874,42.569389
1,214.823959,84.500397,33.630356,17.507677,443.152527,187.583374,31.50979,76.900787,168.345428,178.597610,114.267494,355.517883,210.007874,42.420624
2,214.625107,84.500397,33.630356,17.507677,442.524475,187.519211,31.50979,76.900787,168.252106,178.596573,114.304619,355.537781,210.007874,42.353878
3,214.978348,84.500397,33.630356,17.507677,442.334900,187.508179,31.50979,76.900787,167.905991,178.599899,114.379982,355.753235,210.007874,42.348030
4,215.056732,84.500397,33.630356,17.507677,442.359711,187.507431,31.50979,76.900787,168.016739,178.845871,114.389366,355.979828,210.007874,42.328754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99936,368.681671,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.539398,210.007874,35.086452
99937,368.380188,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.575470,210.007874,35.087753
99938,368.286804,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.612488,210.007874,35.087685
99939,368.123657,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.741150,210.007874,35.087456


## Merging train and test set

In [168]:
df_pred=pd.concat([df_train,df1])
df_pred.columns=MAP
df_pred.reset_index(inplace=True)
df_pred.drop({'index'},axis=1,inplace=True)

## Saving the predictions

In [170]:
df_pred.to_csv('Predicted_values.csv',index=None)